In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 65
WIDTH = 62

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
input_shape=(HEIGHT, WIDTH, 3))

Instructions for updating:
Colocations handled automatically by placer.


/home/laurenz/anaconda3/envs/tf/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [29]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "simple_clock/Datagenerator_simple_clock/Training"
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=0,
      horizontal_flip=False,
      vertical_flip=False,
      width_shift_range=0.03,
      height_shift_range=0.03,
      #shear_range=0.0,
      #zoom_range=0,
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(HEIGHT, WIDTH), batch_size=BATCH_SIZE)

Found 660 images belonging to 60 classes.


In [30]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='sigmoid')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [31]:
FC_LAYERS = [100, 100, 100, 100]
dropout = 0.05

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=60)

In [32]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

NUM_EPOCHS = 50
BATCH_SIZE = 8
num_train_images = 720-60

###optional - loading weights
#finetune_model.load_weights('checkpoints/ResNet50_model_weights.h5')

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
#early_stopping = EarlyStopping(monitor='acc', min_delta=0.002, patience=4, verbose=0, mode='auto', restore_best_weights=True)
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)




# Plot the training and validation loss + accuracy


Epoch 1/50
82/82 [==============================] - 12s 144ms/step - loss: 4.1117 - acc: 0.0168

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/50
82/82 [==============================] - 9s 106ms/step - loss: 4.1114 - acc: 0.0183

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/50
82/82 [==============================] - 9s 106ms/step - loss: 4.1071 - acc: 0.0213

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/50
82/82 [==============================] - 9s 109ms/step - loss: 4.1044 - acc: 0.0137

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/50
82/82 [==============================] - 9s 109ms/step - loss: 4.1017 - acc: 0.0091

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/50
82/82 [==============================] - 9s 110ms/step - loss: 4.0870 - acc: 0.0198

Epoch 00006: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 7/5

KeyboardInterrupt: 

In [ ]:
def plot_training(history):
    acc = history.history['acc']
    #val_acc = history.history['val_acc']
    loss = history.history['loss']
    #val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    #plt.plot(epochs, val_acc, 'r')
    plt.title('Training accuracy')

    plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')

In [ ]:
plot_training(history)